In [15]:
def generate_resonance_structures(mol_list, keep_isomorphic=False, copy=False):
    
    import rmgpy.molecule.filtration as filtration
    
    mol_list_tuple_unfiltered = []
    
    features = resonance.analyze_molecule(mol_list[0])
    method_list = resonance.populate_resonance_algorithms(features)
    
    # Iterate over resonance structures
    min_octet_deviation = min(filtration.get_octet_deviation_list(mol_list))
    index = 0
    while index < len(mol_list):
        molecule = mol_list[index]
        new_mol_list = []

        # Extend methods for molecule only if it is relatively close to the octet rule
        # (don't explore structures that will certainly be filtered out)
        # Since the ground state of the O2/S2/SO species have a higher deviation from the octet rule than some of their
        # excited states, and are not expected to generate an overwhelming amount of resonance structures, they are
        # exempt from this condition.
#         octet_deviation = filtration.get_octet_deviation(molecule)
#         if octet_deviation <= min_octet_deviation or filtration.is_OS(molecule):
        for method in method_list:
            new_mol_list.extend(method(molecule))
#             if octet_deviation < min_octet_deviation:
#                 # update min_octet_deviation to make this criterion tighter
#                 min_octet_deviation = octet_deviation

        for new_mol in new_mol_list:
            # Append to structure list if unique
            for mol in mol_list:
                # for atom in mol.vertices:
                #     if atom.isSulfur() and atom.radicalElectrons == 2 and atom.charge == 1:
                #         logging.info(molecule.toAdjacencyList())
                #         logging.debug(molecule.toAdjacencyList())
                #         raise ValueError('got:\n{0}\ngenerated from:\n{1}'.format(mol.toAdjacencyList(),molecule.toAdjacencyList()))
                if not keep_isomorphic and mol.isIsomorphic(new_mol):
                    break
                elif keep_isomorphic and mol.isIdentical(new_mol):
                    break
            else:
                mol_list.append(new_mol)
                mol_list_tuple_unfiltered.append((new_mol,molecule))

        # Move to the next resonance structure
        index += 1
        
    filtration.filter_structures(mol_list)
    mol_list_tuple = []
    for mol in mol_list:
        for tuple_mol in mol_list_tuple_unfiltered:
            if mol.isIsomorphic(tuple_mol[0]):
                mol_list_tuple.append(tuple_mol)

    return mol_list, mol_list_tuple

In [16]:
import rmgpy.molecule.resonance as resonance
import rmgpy.molecule as molecule
from IPython.display import display

# this is the structure I want to get to:
adj1 = """
multiplicity 4
1 N u0 p1 c0 {2,S} {3,S} {4,S}
2 S u2 p0 c+1 {1,S} {5,D}
3 O u0 p3 c-1 {1,S}
4 O u1 p2 c0 {1,S}
5 O u0 p2 c0 {2,D}
"""

# and this is the structure that gets stuck:
adj2 = """
multiplicity 4
1 S u2 p1 c-1 {2,D} {3,S}
2 O u0 p2 c0 {1,D}
3 N u0 p0 c+1 {1,S} {4,D} {5,S}
4 O u0 p2 c0 {3,D}
5 O u1 p2 c0 {3,S}
"""

mol1 = molecule.Molecule().fromAdjacencyList(adj1)
mol2 = molecule.Molecule().fromAdjacencyList(adj2)

mol_list1, mol_list_tuple1 = generate_resonance_structures([mol1])
mol_list2, mol_list_tuple2 = generate_resonance_structures([mol2])

#find the structures in 1 that aren't included in 2:
unique1 = []
for tuple_mol1 in mol_list_tuple1:
    for tuple_mol2 in mol_list_tuple2:
        if tuple_mol1[0].isIsomorphic(tuple_mol2[0]):
            break
    else:
        unique1.append(tuple_mol1)

#find structures in unique1 that originate from structures in mol_list_tuple2
origin1 = []
for tuple_mol1 in unique1:
    for tuple_mol2 in mol_list_tuple2:
        if tuple_mol1[1].isIsomorphic(tuple_mol2[0]):
            origin1.append(tuple_mol1)



In [19]:
for origin in origin1:
    print 'generated\n{0}from\n{1}'.format(origin[0].toAdjacencyList(),origin[1].toAdjacencyList())

generated
multiplicity 4
1 N u0 p0 c+1 {2,S} {3,D} {4,S}
2 S u2 p1 c-1 {1,S} {5,D}
3 O u0 p2 c0 {1,D}
4 O u1 p2 c0 {1,S}
5 O u0 p2 c0 {2,D}
from
multiplicity 4
1 N u0 p0 c+2 {2,S} {3,S} {4,S}
2 S u2 p1 c-1 {1,S} {5,D}
3 O u0 p3 c-1 {1,S}
4 O u1 p2 c0 {1,S}
5 O u0 p2 c0 {2,D}

